<a href="https://colab.research.google.com/github/rorisDS/workshop_semantic_search/blob/main/Workshop_Buscador_Semantico_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# De Texto a Conocimiento: Construyendo un Buscador Semántico

Este notebook forma parte de la presentación disponible en [este repositorio](https://github.com/rorisDS/workshop_semantic_search). En ella se explora el uso de modelos de NLP para extraer información semántica de textos y realizar búsquedas inteligentes.

Se abordan los principios fundamentales de Machine Learning y NLP, combinando teoría con ejemplos prácticos y código que guían a los asistentes en la implementación de un buscador semántico. El objetivo es aprender desde la generación de embeddings hasta su aplicación en consultas avanzadas.

Este notebook incluye el código necesario para experimentar con los conceptos presentados, acompañado de explicaciones y comentarios que facilitan la comprensión de los aspectos técnicos tratados.

- Autor: [Víctor Manuel Alonso Rorís](https://www.linkedin.com/in/victor-roris/)
- Fecha: 11-03-2025


In [1]:
from IPython.display import clear_output

# Instala las librerias necesarias
!pip install requests
!pip install pymupdf
!pip install faiss-cpu
!pip install python-dotenv
!pip install langchain openai
!pip install -U langchain-community
!pip install openai

# Preinstaladas de serie en google colab
# !pip install transformers
# !pip install sentence-transformers

clear_output() # Limpia todo el log generado en el output!

## Machine Learning

Machine Learning (ML) es una rama de la Inteligencia rtificial que permite a las computadoras aprender patrones a partir de datos sin ser programadas explícitamente para cada tarea. A través de algoritmos, los modelos de ML identifican relaciones en los datos y pueden hacer predicciones o tomar decisiones automáticamente.

El aprendizaje puede ser de distintos tipos, uno de los más importantes es el aprendizaje supervisado en el que se entrena con datos etiquetados.

Una de las técnicas más poderosas dentro de Machine Learning es el uso de Redes Neuronales Artificiales (ANNs). Estas están inspiradas en el funcionamiento del cerebro humano y se componen de capas de neuronas artificiales conectadas entre sí.

### Redes Neuronales Artificaciales

Una red neuronal se compone de capas conectadas entre si, siguiendo una arquitectura. Cada capa recibe una serie de entradas, las procesa aplicando pesos y funciones de activación, y genera una salida que se transmite a la siguiente capa. A medida que el modelo se entrena, ajusta estos pesos para mejorar su precisión en la tarea objetivo.

![red_neuronal](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/neural_network.png)

### Librerias de Machine Learning: PyTorch y TensorFlow

A fin de simplificar el proceso de implementar redes neuronales, han surgido librerías de alto nivel como [PyTorch](https://pytorch.org/tutorials/) y [TensorFlow](https://www.tensorflow.org/), que facilitan la creación, entrenamiento y despliegue de modelos de Machine Learning y Deep Learning.

Ambas librerías permiten aprovechar el poder de las GPU para acelerar el entrenamiento y ofrecen herramientas para gestionar grandes cantidades de datos. Gracias a ellas, desarrollar redes neuronales se ha vuelto más accesible.

![pytorch_meme](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/pytorch_meme.png)


### Let's play

Definimos un modelo (*red neuronal*) simple usando PyTorch

In [ ]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        # Layer 1
        self.linear1 = torch.nn.Linear(100, 200) # Matriz de dimension 100x200
        self.activation = torch.nn.ReLU()        # Function no lineal

        # Layer 2
        self.linear2 = torch.nn.Linear(200, 10)  # Matriz de dimension 200x10
        self.softmax = torch.nn.Softmax()        # Function no lineal

    def forward(self, x):
        # Siendo x vector de dimension: 1x100
        x = self.linear1(x)     # Multiplicacion de matrices: 1x100 * 100x200 = 1x200
        x = self.activation(x)  # f(1x200) => 1x200
        x = self.linear2(x)     # Multiplicacion de matrices: 1x200 * 200x10 = 1x10
        x = self.softmax(x)     # f(1x10) => 1x10
        return x

Instanciamos el modelo que hemos definido y comprobamos que los pesos se asignan incialmente de forma **aleatoria**

In [ ]:
tinymodel = TinyModel()
tinymodel

TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)

In [ ]:
print('Parametros del modelo:')
for param in tinymodel.parameters():
    print(f"\t - Dimension de la capa: {param.shape}")
    print(f"\t - Valores de los parametros: {param}")
    print("\n\n")

Parametros del modelo:
	 - Dimension de la capa: torch.Size([200, 100])
	 - Valores de los parametros: Parameter containing:
tensor([[ 0.0427,  0.0994, -0.0734,  ...,  0.0415, -0.0625,  0.0613],
        [-0.0554, -0.0118, -0.0001,  ...,  0.0147, -0.0258,  0.0939],
        [-0.0311,  0.0081,  0.0371,  ...,  0.0248,  0.0669,  0.0766],
        ...,
        [ 0.0139,  0.0163, -0.0615,  ..., -0.0218, -0.0749, -0.0748],
        [-0.0677,  0.0219,  0.0531,  ...,  0.0869,  0.0377, -0.0512],
        [ 0.0574, -0.0670, -0.0772,  ...,  0.0646, -0.0873, -0.0229]],
       requires_grad=True)



	 - Dimension de la capa: torch.Size([200])
	 - Valores de los parametros: Parameter containing:
tensor([-0.0627,  0.0649,  0.0988, -0.0971, -0.0041,  0.0884, -0.0676, -0.0437,
         0.0489,  0.0051,  0.0232,  0.0711,  0.0736,  0.0445, -0.0290,  0.0744,
        -0.0277,  0.0708,  0.0524,  0.0189,  0.0674, -0.0198, -0.0170,  0.0460,
         0.0736, -0.0003, -0.0416, -0.0202,  0.0437, -0.0665,  0.0063, -0.

Testemos como predicir con un modelo!

In [ ]:
# Generamos (aleatoriamente) un vector de entrada de dimension: 1x100
x = torch.rand(1, 100)
print(f"Vector de entrada ({x.shape}): {x}")

Vector de entrada (torch.Size([1, 100])): tensor([[0.8461, 0.2804, 0.1708, 0.0513, 0.9922, 0.1993, 0.0520, 0.1732, 0.8150,
         0.3280, 0.7469, 0.3274, 0.3221, 0.4022, 0.7565, 0.5800, 0.7555, 0.1149,
         0.2211, 0.5360, 0.1235, 0.6421, 0.9871, 0.9351, 0.3065, 0.4600, 0.3683,
         0.5972, 0.5381, 0.2328, 0.1556, 0.0175, 0.5805, 0.6488, 0.4841, 0.9396,
         0.0231, 0.6507, 0.7362, 0.9249, 0.5027, 0.5500, 0.9413, 0.7240, 0.1126,
         0.6075, 0.7075, 0.8345, 0.6531, 0.4328, 0.6044, 0.5451, 0.2136, 0.5279,
         0.0481, 0.0619, 0.6415, 0.9557, 0.5411, 0.1528, 0.0271, 0.0686, 0.1827,
         0.9039, 0.2553, 0.1875, 0.7328, 0.8153, 0.8465, 0.8949, 0.6930, 0.3558,
         0.2386, 0.8980, 0.8251, 0.8478, 0.5125, 0.6681, 0.0510, 0.5141, 0.6751,
         0.5207, 0.8435, 0.6346, 0.3818, 0.3440, 0.4723, 0.5969, 0.1139, 0.5244,
         0.8299, 0.0970, 0.5150, 0.4072, 0.6778, 0.1953, 0.5814, 0.2957, 0.3282,
         0.5626]])


In [ ]:
# Predecimos y con el modelo actual (parametros asignados aletoriamente)
y = tinymodel(x)
print(f"Vector de salida ({y.shape}): {y}")

Vector de salida (torch.Size([1, 10])): tensor([[0.0928, 0.1255, 0.0984, 0.0777, 0.1166, 0.1168, 0.1125, 0.0708, 0.0993,
         0.0896]], grad_fn=<SoftmaxBackward0>)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Todo modelo puede ser entrenado para ajustar sus pesos o parámetros al conjunto de datos de entrenamiento. Sin embargo, este proceso queda fuera del alcance de este workshop.

Si deseas aprender cómo entrenar un modelo en PyTorch, puedes consultar el siguiente enlace: https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

### Tu Turno  

- **Repite la ejecución del modelo**: Vuelve a instanciar el modelo y ejecuta las operaciones nuevamente para comprobar que los pesos realmente se inicializan de forma aleatoria.  
- **Modifica la arquitectura**: Agrega más capas o cambia la dimensión de las matrices y observa cómo afecta al comportamiento del modelo.  
- **Explora otras funciones de activación**: Investiga y prueba distintas funciones no lineales disponibles en PyTorch: [Documentación de funciones de activación](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity).  

## Tokenizador

Los **tokenizers** son componentes clave en **NLP**.  Su propósito es traducir texto en datos que pueden ser procesados por modelos. Los modelos solo procesan datos numéricos, por lo tanto, el principal objetivo de los tokenizadores es convertir el texto de entrada en **vectores numéricos**.


![tokenizer](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/NLP_tokenizer.png)

### HuggingFace

[**HuggingFace**](https://huggingface.co/docs) es una organización que ofrece herramientas y plataformas de **Inteligencia Artificial**, especialmente centradas en **NLP**.

Entre los servicios ofrecidos por HuggingFace se encuentra el [**Hub de modelos**](https://huggingface.co/docs/hub/index), una plataforma que permite a la comunidad almacenar, descubrir y compartir modelos. Esto facilita el acceso tanto a modelos entrenados por individuos particulares como a modelos desarrollados por la propia organización. Échale un vistazo a como usar el Hub en: https://huggingface.co/docs/hub/models-the-hub

Una de las herramientas más destacadas de HuggingFace es la [**librería Transformers**](https://huggingface.co/docs/transformers/index), que proporciona múltiples mecanismos que nos abstraen de la implementación directa de las funcionalidades de **NLP**, incluida la **tokenización**. Échale un vistazo rápido a como usarla en: https://huggingface.co/docs/transformers/quicktour



### Como funciona un Tokenizado

Un **tokenizador** en **NLP** es una herramienta fundamental que convierte un texto sin procesar en unidades más pequeñas llamadas **tokens**, que pueden ser palabras, subpalabras, caracteres o símbolos especiales. Esta segmentación facilita el análisis, procesamiento y comprensión del texto por parte de los modelos de NLP.

Los algoritmos de **tokenización** dividen los texto en *subpalabras*. Estos algoritmos se basan en el principio de que las palabras de uso frecuente no deben dividirse en subpalabras más pequeñas, pero las palabras raras sí deben descomponerse en subpalabras significativas.

Por ejemplo, *“inesperadamente”* podría considerarse una palabra rara y podría descomponerse en *“inesperado”* y *“mente”*. Estas partes probablemente aparecerán con mayor frecuencia como subpalabras independientes, mientras que al mismo tiempo se conserva el significado de *“inesperadamente”* mediante el significado compuesto de *“inesperado”* y *“mente”*.

Estas subpalabras pueden ser mapeadas a **IDs numéricos**. Un **tokenizador** consta de un **diccionario** que permite la *conversión directa* entre subpalabra e ID. Este diccionario de tokenización actúa simplemente como una tabla de correspondencia entre cada token y un número entero único. Es importante destacar que esta conversión no debe confundirse con un **embedding**, ya que no proporciona ninguna información semántica o contextual sobre el texto. Mientras que un **embedding** representa las relaciones semánticas entre palabras al ubicarlas en un espacio vectorial donde la proximidad indica similitud de significado, la asignación de IDs por parte del **tokenizador** es puramente un proceso de indexación. Cada token recibe un número arbitrario y fijo, sin tener en cuenta el contexto o el significado del token en relación con otros.

Si quieres saber más: https://huggingface.co/learn/nlp-course/chapter2/4

### Let's play!

Seleccionamos un modelo a usar

In [ ]:
# Usando el Hub de modelos de HuggingFace, seleccionamos un model
#  En este caso, un modelo basado en lenguaje español
model_id = "dccuchile/bert-base-spanish-wwm-cased"

# Si quieres, puedes ir al Hub y buscar cualquier otro modelo: https://huggingface.co/models

Instanciamos el `Tokenizador` para el modelo seleccionado

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Escribimos un texto a tokenizar

In [ ]:
sequence = "Aprendiendo como funciona un tokenizer"

Rompemos el texto en tokens

In [ ]:
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Apren', '##diendo', 'como', 'funciona', 'un', 'to', '##k', '##en', '##iz', '##er']


Mapeamos cada token a un id numerico (**conversion directa usando el diccionario interno del Tokenizer!**)

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[12340, 3052, 1184, 6023, 1049, 1166, 981, 1014, 1376, 1015]


Podemos reconvertir los ids numericos a tokens y en consecuencia al texto de partida!

In [ ]:
decoded_string = tokenizer.decode(ids)

print(decoded_string)

Aprendiendo como funciona un tokenizer


### Tu Turno

- **Prueba otros modelos**: Explora diferentes modelos en el [Hub de modelos de HuggingFace](https://huggingface.co/models). ¿Generan los mismos tokens para el mismo texto?  
- **Experimenta con palabras inventadas**: ¿Qué sucede cuando introduces palabras inexistentes o de otros idiomas?  
- **Analiza los primeros tokens**: ¿Cuáles son los tokens representados por los primeros 10 IDs (0-9)? ¿El tokenizador solo representa palabras?  



### Reflexión

Las subpalabras pueden ser mapeadas a IDs numéricos mediante un diccionario de tokenización, el cual actúa simplemente como una tabla de correspondencia entre cada token y un número entero único. Es importante destacar que esta conversión no debe confundirse con un embedding, ya que no proporciona ninguna información semántica o contextual sobre el texto. Mientras que un embedding representa las relaciones semánticas entre palabras al ubicarlas en un espacio vectorial donde la proximidad indica similitud de significado, la asignación de IDs por parte del tokenizador es puramente un proceso de indexación. Cada token recibe un número arbitrario y fijo, sin tener en cuenta el contexto o el significado del token en relación con otros.

## Tareas de NLP

El Procesamiento del Lenguaje Natural (NLP) abarca diversas tareas que permiten a las máquinas comprender, interpretar y generar texto de manera efectiva. Estas tareas utilizan modelos de Machine Learning y redes neuronales para extraer significado y contexto del lenguaje humano.

Algunas de estas tareas se ilustran en la siguiente imagen:

![nlp_tasks](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/nlp_tasks.png)

Para conocer en detalle la naturaleza y funcionamiento de estas tareas, se puede consultar la web de Hugging Face: [Hugging Face Tasks](https://huggingface.co/tasks).

Además, Hugging Face ofrece una amplia colección de modelos preentrenados que cubren estas tareas en su repositorio: [Hugging Face Models](https://huggingface.co/models).


### Let's play

A continuación, se presentan algunos ejemplos de código para realizar diversas tareas de Procesamiento del Lenguaje Natural (NLP) utilizando la biblioteca [Transformers](https://huggingface.co/docs/transformers/quicktour) de Hugging Face. Para ello, se emplearán modelos disponibles en el [Hugging Face Model Hub](https://huggingface.co/models), explorando distintas formas de realizar predicciones mediante modelos, tokenizers y [pipelines](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines).

#### Traduccion

En este ejemplo, vamos a utilizar el directamente `modelo` y el `tokenizer` para generar la prediccion.

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# Modelo disponible a través del hub de hugginface:
#   https://huggingface.co/Helsinki-NLP/opus-mt-en-es
model_key = "Helsinki-NLP/opus-mt-en-es"

# Instancia modelo y tokenizador
model = AutoModelForSeq2SeqLM.from_pretrained(model_key)
tokenizer = AutoTokenizer.from_pretrained(model_key)
model.eval()

# Traducir
sentence = 'flowers are white'
input_ids = tokenizer(sentence, return_tensors='pt')
outputs = model.generate(**input_ids)
# La salida del modelo son los tokens que deben ser reconvertidos a texto mediante el tokenizador
outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(outputs)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


las flores son blancas


#### Classificacion de texto

En este caso, vamos a probar el uso de un `pipeline` especifico de clasificacion de texto para abstraer la ejecucion del `tokenizer` y el `modelo`.

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TextClassificationPipeline

# Modelo disponible a través del hub de hugginface:
#   https://huggingface.co/M47Labs/spanish_news_classification_headlines
model_key = "M47Labs/spanish_news_classification_headlines"

# Instancia tokenizador, modelo y pipeline
tokenizer = AutoTokenizer.from_pretrained(model_key)
model = AutoModelForSequenceClassification.from_pretrained(model_key)
nlp = TextClassificationPipeline(model = model, tokenizer = tokenizer)

text_noticia = 'Reducir emisiones es imprescindible a fin de favorer la regeneracion del ecosistema'
print(nlp(text_noticia))

text_noticia = "Un partido reñido que se soluciono en la prórroga por un penalty"
print(nlp(text_noticia))


tokenizer_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'medio_ambiente', 'score': 0.7891209125518799}]


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

[{'label': 'deportes', 'score': 0.97039794921875}]


#### Resumir

Finalmente, demostramos como usar el `pipeline` directamente para la ejecucion de la prediccion.

In [ ]:
from transformers import pipeline

# Modelo disponible a través del hub de hugginface:
#   https://huggingface.co/tennessejoyce/titlewave-t5-base
model_key = "tennessejoyce/titlewave-t5-base"

classifier = pipeline('summarization', model=model_key)
article = """In this workshop, participants will explore Neural Networks, NLP, and embeddings to build a semantic search engine using Python. They will learn how neural networks process text, how embeddings capture meaning, and how to use pre-trained models to convert text into vector representations. Using Hugging Face Transformers, TensorFlow, and FAISS, they will implement a search function that retrieves results based on semantic similarity rather than keywords. By the end, attendees will have a working NLP-powered search engine and a solid understanding of modern language models and their applications. No prior experience required!"""
classifier(article)

config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


[{'summary_text': 'How to build a semantic search engine using Python?'}]

#### Tu turno

* **Prueba otras tareas de NLP por tu cuenta**

## Embeddings

El embedding de un texto es su representación en un espacio vectorial de alta dimensionalidad. Este embedding se expresa como un vector numérico que captura relaciones semánticas y contextuales del texto. En este espacio, textos con significados similares tienden a ubicarse en posiciones cercanas, permitiendo medir similitud.

![embedding_relaciones](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/embeddings_relations.png)

El uso de modelos de NLP permite generar embeddings que capturan relaciones semánticas entre palabras, frases o documentos. Modelos como `Word2Vec`, `GloVe`, `FastText` y `Transformers` (ej. `BERT`, `Sentence Transformers`) entrenan estos embeddings para que reflejen similitudes de significado, incluso cuando las palabras no coinciden exactamente. Estos vectores pueden utilizarse en tareas como búsqueda semántica, clasificación de texto, recomendación de contenido y clustering, permitiendo que los algoritmos comprendan mejor el lenguaje y encuentren conexiones más allá de simples coincidencias de palabras.

### Sentence Tranformers

[Sentence Transformers](https://sbert.net/) es una biblioteca de Python basada en HuggingFace que abstrae y simplifica la gestión de modelos para la generación de embeddings. Es decir, es una libreria diseñada para generar representaciones vectoriales de textos mediante modelos de deep learning optimizados para tareas de búsqueda semántica, clasificación de texto, clustering y más. Es una libreria mantenida por Hugging Face.

### Let's play

Comprobemos como poder obtener embeddings usando modelos de NLP.

In [ ]:
# Modelo disponible a través del hub de hugginface:
# https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
model_key = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

In [ ]:
# Frases a generar los embeddings
sentences = ["Hola", "Buenos dias", "Adios"]

La obtención del embedding usando transformers es algo compleja, ya que implica
varios cálculos sobre el vector de salida del modelo.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Carga el modelo a partir del HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(model_key)
model = AutoModel.from_pretrained(model_key)
model.eval()

# Tokeniza las frases
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Calcula los embeddings de los token
model_output = model(**encoded_input)

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Agrupa los embeddings de los tokens para obtener el embedding de las frases
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Embeddings de las frases:")
print(sentence_embeddings)

sentence_embeddings.shape

Embeddings de las frases:
tensor([[-0.0170, -0.0247, -0.0130,  ...,  0.2453,  0.0008, -0.1304],
        [-0.0581, -0.0641, -0.0164,  ...,  0.1326,  0.0073, -0.0602],
        [-0.0313,  0.1261, -0.0146,  ...,  0.1787,  0.1019, -0.0816]],
       grad_fn=<DivBackward0>)


torch.Size([3, 768])

`Sentence Transformers` permite abstraer la complejidad de `transformers` para la obtención de los embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer

# Carga el modelo a partir del HuggingFace Hub
model = SentenceTransformer(model_key)
sentence_embeddings = model.encode(sentences)

print("Embeddings de las frases:")
print(sentence_embeddings)

Embeddings de las frases:
[[-0.01695467 -0.02471214 -0.01296082 ...  0.24528737  0.00076534
  -0.13042565]
 [-0.05806712 -0.06408359 -0.01635563 ...  0.13261172  0.00730249
  -0.06017587]
 [-0.03130954  0.1261161  -0.01455805 ...  0.1787395   0.10191587
  -0.0815663 ]]


### Tu Turno  

- **Orden de las palabras**: Comprueba si el embedding de *"el coche rojo"* y *"el rojo coche"* es el mismo. ¿El modelo captura la diferencia en el orden?

- **Prueba otros modelos**: Explora diferentes modelos en el [Hub de modelos de HuggingFace](https://huggingface.co/models). ¿Para los mismos textos devuelven los mismos embeddings?  

- **Longitud del texto**: Genera embeddings para frases de distinta longitud que expresen la misma idea (ej. *"El perro duerme"* vs. *"El perro está durmiendo plácidamente en su cama"*). ¿Cómo afecta la cantidad de palabras al embedding?  

- **Palabras fuera de vocabulario**: Prueba con palabras inventadas o muy poco comunes. ¿El modelo genera un embedding o devuelve algo diferente?  

## Calcular distancias entre embeddings

A fin de comparar la relación entre dos embeddings, se utilizan medidas de distancia o similitud. La métrica más común en NLP es la **Similitud del Coseno**, que mide el ángulo entre dos vectores en el espacio.

La **Similitud del Coseno** se calcula como:

![](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/cosine_similarity.png)

donde 𝐴 y 𝐵 son los vectores de los embeddings, 𝐴⋅𝐵 es su producto punto, y ∥𝐴∥ y ∥𝐵∥ son sus normas (magnitudes). Su valor varía entre -1 y 1, donde 1 indica máxima similitud, 0 indica que los vectores son ortogonales (sin relación), y -1 indica oposición total.

Esta métrica es ideal en NLP porque captura similitud independientemente de la magnitud del vector, centrándose solo en la orientación semántica.




### Let's play

In [ ]:
from sentence_transformers import SentenceTransformer

sentences = ["Hola", "Buenos dias", "coche rojo"]

# Modelo disponible a través del hub de hugginface:
# https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
model_key = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model = SentenceTransformer(model_key)
sentence_embeddings = model.encode(sentences, convert_to_numpy=True)

In [ ]:
embedding1 = sentence_embeddings[0]
embedding2 = sentence_embeddings[1]
embedding3 = sentence_embeddings[2]

Calculo de la similitud del coseno usando NumPy

In [ ]:
import numpy as np

dot_product = np.dot(embedding1, embedding2)
magnitude_1 = np.linalg.norm(embedding1)
magnitude_2 = np.linalg.norm(embedding2)

cosine_similarity = dot_product / (magnitude_1 * magnitude_2)
print(f"Similitud del coseno usando NumPy: {cosine_similarity}")

Similitud del coseno usando NumPy: 0.7888635396957397


Calculo de la similitud del coseno usando scikit-learn

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_result = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))
print(f"Similitud del coseno usando scikit-learn: {cosine_similarity_result[0][0]}")

Similitud del coseno usando scikit-learn: 0.7888635396957397


Calculo de la similitud del coseno usando sentence-transformers

In [ ]:
similarity = model.similarity(embedding1, embedding2)
print(f"Similitud del coseno usando sentence-transformers: {similarity.tolist()[0][0]}")

Similitud del coseno usando sentence-transformers: 0.7888635396957397


Compara la similitud entre diferentes palabras con diferente semántica

In [ ]:
print(f"Similitud entre frases '{sentences[0]}' y '{sentences[1]}': {model.similarity(sentence_embeddings[0], sentence_embeddings[1]).tolist()[0][0]}")
print(f"Similitud entre frases '{sentences[0]}' y '{sentences[2]}': {model.similarity(sentence_embeddings[0], sentence_embeddings[2]).tolist()[0][0]}")

Similitud entre frases 'Hola' y 'Buenos dias': 0.7888635396957397
Similitud entre frases 'Hola' y 'coche rojo': 0.23927712440490723


### Tu turno

- **Similitud entre frases**: Calcula la similitud entre los embeddings de frases con significados similares, como *\"El clima es agradable hoy\"* y *\"Hace buen tiempo\"*. ¿Qué valores obtienes?  

- **Sinónimos y antónimos**: Compara la distancia entre los embeddings de sinónimos y antónimos. ¿Los resultados son coherentes con su significado?  

- **Días de la semana**: Calcula la distancia entre los embeddings de los días de la semana. ¿Notas algún patrón?  

- **Prueba otros modelos**: Explora diferentes modelos en el [Hub de modelos de HuggingFace](https://huggingface.co/models). ¿Para las mismas palabras obtienes la misma similitud?  

- **Comparación entre idiomas**: Obtén los embeddings de frases en distintos idiomas, como *\"Hello, how are you?\"* y *\"Hola, ¿cómo estás?\"*. ¿Son similares si el modelo es multilingüe?  

- **Explora otras métricas**: Prueba distintas métricas de distancia o similitud de embeddings y compara los resultados (consulta las implementaciones en [sklearn](https://scikit-learn.org/stable/api/sklearn.metrics.html#module-sklearn.metrics.pairwise)).  


## Dense search

Una vez que representamos el texto como embeddings, necesitamos una forma eficiente de buscar en grandes volúmenes de datos. En lugar de comparar cada vector uno por uno, lo cual sería muy costoso, utilizamos bases de datos de vectores que optimizan esta búsqueda mediante estructuras especializadas.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/75/Base_de_datos_de_vectores_o_vectorial.jpg/1024px-Base_de_datos_de_vectores_o_vectorial.jpg" alt="base_datos_vectores" width="700"/>

Herramientas como [FAISS](https://ai.meta.com/tools/faiss/) (de Meta) o [Annoy](https://github.com/spotify/annoy) (de Spotify) permiten realizar búsquedas de similitud basadas en la distancia del coseno (o métricas similares) de forma rápida y escalable. Estas bases de datos permiten realizar **Dense Search**, es decir, búsquedas en espacios densos de embeddings.

### Faiss

[FAISS](https://ai.meta.com/tools/faiss/) (Facebook AI Similarity Search) es una biblioteca de código abierto desarrollada por Meta para realizar búsquedas de similitud en grandes colecciones de embeddings de manera eficiente. Está optimizada para manejar millones o incluso miles de millones de vectores, permitiendo encontrar rápidamente los más similares a una consulta dada.

FAISS logra esta eficiencia utilizando estructuras como:

* **Índices aproximados**: En lugar de comparar cada vector individualmente, FAISS usa técnicas como clustering y cuantización para reducir la cantidad de cálculos sin comprometer demasiado la precisión.
* **Soporte para GPU**: Acelera la búsqueda de vecinos más cercanos utilizando procesamiento en paralelo.
* **Diferentes métricas de distancia**: Como la distancia del coseno, distancia euclidiana y otras, adaptándose a distintas aplicaciones.

Gracias a estas características, FAISS es ampliamente utilizado en buscadores semánticos, sistemas de recomendación y otras aplicaciones que requieren búsqueda eficiente en espacios de alta dimensión.

### Let's play

In [2]:
from sentence_transformers import SentenceTransformer
from typing import List

class Embedder:

  def __init__(self, model_key: str = None):
    if model_key is None:
      # Modelo disponible a través del hub de hugginface:
      # https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2
      model_key = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    self.model = SentenceTransformer(model_key, trust_remote_code=True)

  def embed_passages(self, passages: List[str]):
    return self.model.encode(passages, convert_to_numpy=True)

  def embed_query(self, query: str):
    # Some models can have a different function to encode queries!
    return self.model.encode(query, convert_to_numpy=True)

embedder = Embedder()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Indexar passages
passages = [
    # Ruido
    "hola", "buenos dias", "coche rojo", "workshop", "la estufa esta encendida",
    "Pienso, luego existo",
    # Textos sobre comida
    "la comida del campus sabe a carton",
    "comer en un restaurante de estrella michelin es digno de estudio",
    "la comida de mi madre deberia estudiarse en la universidad"
]

passage_embeddings = embedder.embed_passages(passages)

Construimos el indice

In [9]:
import faiss

# dimension de los embeddings
d = passage_embeddings[0].shape[0]
print(f"Dimension de los embeddings: {d}")

# Construimos un indice para embeddings de dimension d
index = faiss.IndexFlatL2(d)

Dimension de los embeddings: 768


In [10]:
# Indexamos los embeddings en el indice
index.add(passage_embeddings)

In [16]:
# Generate an embedding for the query
query = ["¿Cómo se come en la universidad?"]

query_embedding = embedder.embed_query(query=query)

k = 4  # Retrieve the k closest neighbors
distances, indices = index.search(query_embedding, k)

# Display the results
for it, (query_indice, query_distance) in enumerate(zip(indices, distances)):
  print(f"Query: {query[it]}")
  print()
  for idx, dist in zip(query_indice, query_distance):
      print(f"Index: {idx}, Distance: {dist}")
      print("Text: ", passages[idx])
      print()
  print("--------------\n")

Query: ¿Cómo se come en la universidad?

Index: 6, Distance: 3.8526031970977783
Text:  la comida del campus sabe a carton

Index: 8, Distance: 4.105026721954346
Text:  la comida de mi madre deberia estudiarse en la universidad

Index: 7, Distance: 6.69992733001709
Text:  comer en un restaurante de estrella michelin es digno de estudio

Index: 1, Distance: 10.642995834350586
Text:  buenos dias

--------------



### Tu turno

- **Modifica la consulta**: Cambia la consulta por algo completamente diferente (por ejemplo, *\"¿De qué color es el coche?\"* o *\"¿Qué electrodoméstico está funcionando?\"*). ¿Siguen siendo relevantes los fragmentos recuperados? ¿Cómo cambian las distancias?

- **Aumenta el número de vecinos**: Modifica el valor de `k` para recuperar más (o menos) vecinos cercanos. ¿Se vuelven menos relevantes los resultados a medida que aumenta `k`?

- **Añade más fragmentos**: Expande la lista de `passages` con **al menos 10 nuevas entradas** sobre diferentes temas (por ejemplo, tecnología, deportes, música, historia). ¿Siguen siendo relevantes los resultados o los datos adicionales introducen ruido?

- **Experimenta con múltiples consultas**: Modifica el código para **manejar múltiples consultas simultáneamente** (por ejemplo, `query=["¿Cómo cocina mi mamá?", "¿Cómo va tu conexión a Internet?"]`). ¿Cómo responde el índice a múltiples búsquedas?

- **Usa un modelo diferente**: Sustituye el modelo actual por otro del [Hugging Face Model Hub](https://huggingface.co/models). ¿Observas alguna mejora o degradación en la precisión de la búsqueda?

## Construyendo un Buscador Semántico

Un buscador semántico permite recuperar información relevante basándose en el significado del texto, en lugar de depender únicamente de coincidencias exactas de palabras clave. Para lograr esto, convertimos los textos y las consultas en embeddings y utilizamos una base de datos de vectores para encontrar los más similares.

![buscador_semantico](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/buscsemant.png)

La construcción de un buscador semántico consta de dos etapas principales:

* **Indexación**: En esta fase, los materiales de referencia se convierten en embeddings y se almacenan en una base de datos de vectores, como FAISS, para facilitar la recuperación eficiente.
* **Búsqueda**: Cuando se introduce una consulta, se genera su embedding y se compara con los documentos indexados para recuperar los más relevantes según su similitud semántica.

A continuación, exploraremos en detalle cada una de estas etapas y cómo implementarlas en la práctica.

### Indexación

Este proceso se divide en varias fases dentro de la ingestión de datos (ETL):

1. **Extracción de Texto de Documentos**: El primer paso es obtener el contenido de los documentos a indexar. Estos documentos pueden estar en diferentes formatos, como PDF, DOCX o TXT. Se utilizan herramientas como PyMuPDF para extraer el texto de documentos PDF, garantizando que toda la información relevante sea capturada para su posterior procesamiento.

2. **División del Texto en Fragmentos (Chunking)**: Los documentos suelen ser extensos, por lo que es necesario dividirlos en partes más pequeñas llamadas passages o fragmentos. Esto mejora la eficiencia de la búsqueda, permitiendo recuperar respuestas más precisas. Se puede hacer dividiendo el texto en bloques de un número fijo de palabras o frases.

3. **Conversión de los Fragmentos en Embeddings**: Cada fragmento de texto se convierte en un embedding, que es una representación numérica en un espacio multidimensional. Para ello, se emplean modelos de aprendizaje profundo, como Sentence Transformers. Estos modelos convierten el significado del texto en vectores, permitiendo comparaciones basadas en similitud semántica en lugar de coincidencia exacta de palabras.

4. **Indexación en una Base de Datos de Vectores**: Los embeddings generados se almacenan en una base de datos especializada en búsquedas vectoriales, como FAISS (Facebook AI Similarity Search). FAISS permite realizar búsquedas rápidas y eficientes utilizando técnicas avanzadas de comparación de vectores, identificando los fragmentos más relevantes para una consulta dada.


![indexation](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/buscsemant_indexation.png)

In [ ]:
import requests
import fitz  # PyMuPDF
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

def download_pdf(url, output_path: str):
    """Descarga un PDF de una url"""
    response = requests.get(url)
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"PDF descargado: {output_path}")

def pdf_to_text(pdf_path: str) -> str:
    """Convierte el contenido de un PDF a texto"""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def split_text(text: str, chunk_size: int=200) -> list:
    """Rompe el texto en trozos"""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def create_embeddings(passages: list, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """Convierte textos a embeddings"""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(passages, convert_to_numpy=True)
    return embeddings

def store_embeddings(embeddings, index_path="faiss_index.bin"):
    """Indexa embeddings en Faiss"""
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    faiss.write_index(index, index_path)
    print(f"Embeddings guardados en {index_path}")

In [ ]:
# 0. Obtengo los documentos a procesar
pdf_url = "https://d1.awsstatic.com/aws-analytics-content/OReilly_book_Natural-Language-and-Search_web.pdf"  # URL de ejemplo
pdf_path = "documento.pdf"
download_pdf(pdf_url, pdf_path)

# 1. Extraccion de texto
text = pdf_to_text(pdf_path)

# 2. Division en fragmentos
passages = split_text(text)

# 3. Genera embeddings
embeddings = create_embeddings(passages)

# 4. Guarda en la base de datos de vectores
index_path = "faiss_index.bin"
store_embeddings(embeddings, index_path)
print("Indexación completada.")

PDF descargado: documento.pdf
Embeddings guardados en faiss_index.bin
Indexación completada.


### Búsqueda

Una vez que los documentos han sido indexados y almacenados en una base de datos de vectores, el buscador semántico puede responder a consultas del usuario de manera eficiente. Este proceso se divide en varias fases:

1. **Conversión de la Consulta a Embedding**:
Cuando un usuario ingresa una consulta, el sistema primero la convierte en un embedding. Para ello, se utiliza el mismo modelo de transformación que se usó para indexar los documentos (por ejemplo, Sentence Transformers). Esta conversión permite representar el significado de la consulta en un espacio vectorial, similar a cómo se representaron los documentos en la fase de indexación.

2. **Búsqueda del Embedding en la Base de Datos de Vectores**:
Una vez obtenida la representación numérica de la consulta, se compara con los embeddings almacenados en la base de datos vectorial (como FAISS). FAISS utiliza algoritmos de búsqueda eficiente para encontrar los vectores más cercanos en términos de similitud.

3. **Identificación de los Fragmentos Más Similares**:
Una vez identificados los embeddings más similares, se utiliza su identificador para recuperar cada passage. Finalmente, al usuario se le presenta el contenido textual correspondiente como respuesta.

![busqueda](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/buscsemant_query.png)

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

def load_index(index_path="faiss_index.bin"):
    index = faiss.read_index(index_path)
    return index

def search(query, index, passages, top_k=5):

    # 1. Convierte la query a embedding
    query_embedding = create_embeddings([query])

    # 2. Busca los top_k resultados mas similares
    distances, indices = index.search(query_embedding, top_k)

    # 3. Identifica los passages y su distancia asociada
    results = [
      (passages[idx], distances[0][i]) for i, idx in enumerate(indices[0])
    ]

    return results


In [ ]:
# Cargar el índice y los textos
index_path = "faiss_index.bin"

index = load_index(index_path)

# Realizar búsqueda
test_query = "How many dimensions should an embedding have?"
results = search(test_query, index, passages)

print("Resultados de búsqueda:")
for passage, score in results:
    print(f"Score: {score:.4f}\n{passage}\n")

Resultados de búsqueda:
Score: 1.2135
helps reduce nonsensical implications, but how many more dimen‐ sions is enough? 14 | Chapter 3: Vectors: Representing Semantic Information Dimensionality One answer is to use as many dimensions as there are words in the vocabulary. Let aardvark = (1,0,0,0,...), abacus = (0,1,0,0,...), apple = (0,0,1,0,...), and so on. With this encoding, no two words can be added to give another word, since no word’s representation can have more than one 1. This is often called sparse, keyword, or one-hot encoding, and the corresponding vectors are known as sparse or one-hot vectors. Sparse vectors are quite useful in search and other natural language tasks, and until quite recently they were considered to be the state of the art. But sparse vectors can’t combine in a meaningful way (no word has more than one “1”). Take the words red and fruit. Adding their vectors should not produce the vector corresponding to aardvark or banana. But it would make sense if it equ

### Tu turno

* **Añade nuevos materiales**: Busca más referencias (ej., libros, artículos) e indexa su contenido para realizar consultas sobre ellos.

* **Evalúa su desempeño en dominios especializados**: Prueba con textos de nicho, como documentación legal o artículos científicos. Si el buscador no arroja buenos resultados, ¿qué factor lo está limitando? ¿Es FAISS? ¿El modelo de embeddings? ¿El tamaño de los chunks? ¿La extracción del texto?

* **Prueba diferentes modelos**: Explora el [Hub de modelos de HuggingFace](https://huggingface.co/models) y prueba alternativas. ¿Los resultados son consistentes para las mismas consultas?

* **Ajusta el tamaño de los chunks**: Durante la indexación, experimenta con diferentes tamaños de fragmentos de texto y evalúa cómo afectan los resultados.

* **Explora otras bases de datos de vectores**: ¿Cómo cambia el rendimiento si usas Annoy u otra alternativa en lugar de FAISS?


El código presentado en este notebook es un punto de partida básico. En un caso real, deberías considerar que:

* **Los materiales de referencia pueden provenir de múltiples fuentes y formatos** (PDFs, documentos, páginas web, bases de datos, etc.).

* **La indexación debe ser dinámica**: No siempre se indexa todo al mismo tiempo; nuevos documentos pueden añadirse posteriormente.

* **Es clave gestionar metadatos**: FAISS solo devuelve un identificador numérico para cada chunk de texto, por lo que necesitas almacenar metadatos que te permitan localizar el documento original y la posición exacta del fragmento indexado.

Muchas otras optimizaciones pueden mejorar el rendimiento y la robustez del sistema. Si quieres construir un buscador semántico real, ¡te toca seguir explorando y mejorándolo!

    

## Aplicación de un Buscador Semántico: RAG

El concepto de RAG (Retrieval-Augmented Generation) combina dos componentes: recuperación y generación. En lugar de generar una respuesta solo a partir del conocimiento internalizado en el LLM, el modelo primero recupera información relevante de una base de datos externa o de un índice de búsqueda (como un buscador semántico), y luego usa esta información recuperada para generar una respuesta más precisa y contextualizada.

Este enfoque mejora la capacidad de los LLMs para responder preguntas complejas y especializadas al acceder a información en tiempo real, haciendo que el sistema sea más potente y adaptable a una amplia gama de tareas. Así, RAG combina lo mejor de la búsqueda semántica con la generación de texto, optimizando tanto la precisión como la relevancia de las respuestas.

![esquema_rag](https://raw.githubusercontent.com/rorisDS/workshop_semantic_search/refs/heads/main/images/rag.png)



### LangChain

[LangChain](https://python.langchain.com/docs/tutorials/) es una librería de código abierto diseñada para facilitar la creación de aplicaciones que integran Large Language Models (LLMs) con diversas fuentes de datos externas, como bases de conocimiento, bases de datos o **buscadores semánticos**. LangChain permite construir flujos de trabajo complejos y personalizados para tareas como la recuperación de información, generación de texto y interacción con APIs (como la ofrecida por OpenAI).  

LangChain simplifica el desarrollo de aplicaciones potentes y escalables, ayudando a crear sistemas de búsqueda y generación de texto que aprovechan el poder de los LLMs de manera eficiente.

### OpenAI

OpenAI es una organización que desarrolla modelos avanzados para generar y comprender texto, imágenes, y más. A través de su API, OpenAI ofrece acceso a potentes modelos como GPT-3 y GPT-4 y facilita la integración de estos modelos en aplicaciones.

La API de OpenAI es accesible través de solicitudes HTTP. Para su uso, es necesario obtener una clave API que se puede generar en el sitio web de OpenAI. Para saber más visita el siguiente enlace: https://platform.openai.com/docs/quickstart



### Let's play!

* OpenAI API KEY

Indica tu OpenAI API Key para poder usar la API de OpenAI (cómo obtener la API key: https://platform.openai.com/docs/quickstart)

In [ ]:
# Indica la API Key de OpenAI
openai_api_key = "TU_API_KEY_DE_OPENAI"

# --- Este proceso es para mi para no compartir mi token con todo el mundo mientras hago el workshop!!!
from google.colab import userdata
try:
  secret_openai_api_key = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
  secret_openai_api_key = None  # Assign None if secret is not found

if secret_openai_api_key is not None:
  openai_api_key = secret_openai_api_key

import os
# LangChain toma el API KEY de las variables de entorno
os.environ["OPENAI_API_KEY"] = openai_api_key

* Sin RAG

Lanzamos una consulta a OpenAI sin contexto

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)

# Prueba 1: Pregunta sobre un documento técnico
# query = "¿Cúal es la cuenta y password por defecto del router Livebox Fibra? Di 'No lo sé' si no sabes la respuesta"  # Si quieres forzar que no se invente la respuesta
query = "¿Cúal es la cuenta y password por defecto del router Livebox Fibra?"
response = chat.predict(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response)
print("\n-----\n")

# Prueba 2: Pregunta sobre un juego de mesa
# query = "Describe como se gana al juego de mesa de Juego de Tronos?  Di 'No lo sé' si no sabes la respuesta"  # Si quieres forzar que no se invente la respuesta
query = "Describe cómo se gana al juego de mesa de Juego de Tronos?"
response = chat.predict(query)
response = chat.predict(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response)
print("\n-----\n")

# Prueba 3: Pregunta sobre un libro
#  query = "¿Quíen es el padre de Marquitos y Lucas? Di 'No lo sé' si no sabes la respuesta"  # Si quieres forzar que no se invente la respuesta
query = "¿Quíen es el padre de Marquitos y Lucas?"
response = chat.predict(query)
response = chat.predict(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response)
print("\n-----\n")

Pregunta: ¿Cúal es la cuenta y password por defecto del router Livebox Fibra?
Respuesta del modelo: La cuenta y contraseña por defecto del router Livebox Fibra de Orange es:

- Usuario: admin
- Contraseña: admin

-----

Pregunta: Describe cómo se gana al juego de mesa de Juego de Tronos?
Respuesta del modelo: El juego de mesa de Juego de Tronos se gana al ser el primer jugador en alcanzar el objetivo de puntos de victoria establecido al comienzo de la partida. Los puntos de victoria se obtienen a través de diferentes acciones, como controlar ciudades y castillos, completar misiones, formar alianzas con otros jugadores y ganar batallas.

Para lograr esto, los jugadores deben planificar estratégicamente sus movimientos, negociar con otros jugadores, formar alianzas temporales y tomar decisiones tácticas en cada turno. Además, es importante tener en cuenta las cartas de personaje y eventos que pueden influir en el desarrollo de la partida.

En resumen, se gana al juego de mesa de Juego de

* Con RAG

Incluyendo contexto para la consulta de OpenAI mediante una búsqueda semántica

In [ ]:
import requests
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


pdf_urls = [
    # Manual instrucciones router Livebox Fibra
    "https://clementecervantesbustos.wordpress.com/wp-content/uploads/2019/05/manual-usuario.pdf",

    # Libro reglas juego de mesa de Juego de Tronos
    "https://zacatrus.es/media/instrucciones/Juego_de_%20Tronos_Reglas_Zacatrus.pdf",

    # Cuento breve de Mario Benedetti
    "https://www.ingenieria.unam.mx/dcsyhfi/material_didactico/Literatura_Hispanoamericana_Contemporanea/Autores_B/BENEDETTI/breve.pdf"
]

# Descargar PDFs
pdf_files = []
for pdf_url in pdf_urls:
  pdf_file = pdf_url.split("/")[-1]
  response = requests.get(pdf_url)
  with open(pdf_file, "wb") as f:
      f.write(response.content)
  pdf_files.append(pdf_file)

# Convertir PDF a texto usando PyMuPDF
texts = []
for pdf_file in pdf_files:
  doc = fitz.open(pdf_file)
  text = "\n".join([page.get_text() for page in doc])
  texts.append(text)

# Dividir texto en trozos (chunks) usando LangChain!
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = []
for text in texts:
  doc_chunks = text_splitter.split_text(text)
  chunks.extend(doc_chunks)

# Crear embedder del documento usando HuggingFace
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embedder = HuggingFaceEmbeddings(model_name=model_name)

# Crear el store vectorial FAISS usando la implementacion de LangChain!
vector_store = FAISS.from_texts(chunks, embedder)

<ipython-input-4-c788bb77e57f>:44: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


# Configurar el LLM (Language Model) de OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# Configurar la cadena RetrievalQA (LangChain) combinando búsqueda semántica con generación
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True
)


# Prueba 1: Pregunta sobre un documento técnico
query = "¿Cúal es la cuenta y password por defecto del router Livebox Fibra?"
response = rag_chain(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response["result"])
print("\n-----\n")

# Prueba 2: Pregunta sobre un juego de mesa
query = "Describe cómo se gana al juego de mesa de Juego de Tronos?"
response = rag_chain(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response["result"])
print("\n-----\n")

# Prueba 3: Pregunta sobre un libro
query = "¿Quíen es el padre de Marquitos y Lucas?"
response = rag_chain(query)
print("Pregunta:", query)
print("Respuesta del modelo:", response["result"])
print("\n-----\n")

Pregunta: ¿Cúal es la cuenta y password por defecto del router Livebox Fibra?
Respuesta del modelo: La cuenta por defecto del router Livebox Fibra es "admin" y la contraseña por defecto es la misma que la de las redes Wi-Fi del router, la cual se puede encontrar en una pegatina alojada bajo el dispositivo, tras el texto CLAVE WIFI.

-----

Pregunta: Describe cómo se gana al juego de mesa de Juego de Tronos?
Respuesta del modelo: En el juego de mesa de Juego de Tronos, los jugadores compiten por el control de las tierras de Poniente. Para ganar, un jugador debe alcanzar un total de 7 puntos de victoria. Estos puntos se obtienen principalmente a través de la posesión de territorios y el control de los tres medidores de influencia del tablero: el Trono de Hierro, los Feudos y la Corte del Rey. Los jugadores pueden obtener puntos de victoria al final de cada ronda de juego, y el primer jugador en alcanzar los 7 puntos de victoria gana la partida.

-----

Pregunta: ¿Quíen es el padre de Mar

### Tu turno  

- **Prueba consultas de conocimiento general**: Formula preguntas cuyo contexto forme parte del conocimiento común, por ejemplo: *\"¿Cuál es la capital de España?\"*. ¿Necesitas RAG para obtener una buena respuesta o el modelo por sí solo puede contestarlas?  

- **Crea preguntas específicas**: Analiza el material de referencia que has indexado y elabora preguntas cuya respuesta solo pueda encontrarse en ese contenido. Comprueba cómo responde el sistema **con** y **sin RAG**.  

- **Integra nuevas fuentes de información**: Añade al índice nuevos documentos obtenidos de la web. Por ejemplo, puedes usar [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para extraer contenido de páginas HTML.  

- **Explora diferentes modelos de embedding**: Revisa el [Hub de modelos de HuggingFace](https://huggingface.co/models) y prueba otros modelos de embeddings. ¿Qué diferencias observas en las respuestas?  

- **Explora diferentes modelos de OpenAI**: Revisa el [Hub de modelos de OpenAI](https://platform.openai.com/docs/models) y prueba otros LLMs cambiando el nombre del modelo en `ChatOpenAI(model_name="xxxxx", temperature=0.0)`. ¿Qué diferencias observas en las respuestas?  

- **Ajusta los parámetros del sistema**: Experimenta con distintos valores de `chunk_size`, `chunk_overlap` y `k` (número de documentos recuperados). Reflexiona sobre cómo estos parámetros afectan la calidad de las respuestas.  

- **Ajusta los parámetros de OpenAI**: Experimenta con distintos valores entre 0 y 1 para el parámetro `temperature` en `ChatOpenAI(model_name="gpt-3.5-turbo", temperature=x.x)`. ¿Cómo afecta este parámetro a las respuestas?  






## Conclusión  

Has llegado al final! A lo largo de este notebook hemos explorado herramientas y conceptos fundamentales para comenzar a trabajar con modelos de lenguaje y construir un buscador semántico básico.  

Te animo a seguir aprendiendo y profundizando en el mundo del NLP y la inteligencia artificial, donde las posibilidades son enormes y en constante evolución.  

No olvides revisar la presentación disponible en [el repositorio](https://github.com/rorisDS/workshop_semantic_search).

Notebook compuesto por [Víctor Manuel Alonso Rorís](https://www.linkedin.com/in/victor-roris/) a 11 de Marzo de 2025